# NCAA Man's, Gradient Boosted Trees (XGBoost)

## Work In Progress, Stay Tune...
**Notebook Strategy...**

Hello After studing multiple notebook I want to describe the strategy I will follow.
This are the steps:

* Load the Seed, Season Results and Tournament Results information.
* Build aggregated features by Team and Season from the Season Results Dataset.
* Merge the Information Seed and Aggregated information into the Tournament Results Dataset by Team and Season
* Use the Tournament Results Dataset with the outcomes of the game to construct a target variable
* The target variable will be the outcome of the game for Team A, Win = 1 Lost = 0

**Updates**

02/24/2022: First baseline model constructed using an XGBoost Classifier.

**Notebook Used**
* https://www.kaggle.com/theoviel/using-last-year-s-2nd-place
* https://www.kaggle.com/theoviel/ncaa-starter-the-simpler-the-better

Both Notebooks are from **theoviel** so thanks so much to the author.

**Experiment Summary**

**XGBoost, Data Range: 2016-2021 Added Feature: PointsRatio**
* CV Score: 0.60800
* LB Score: 0.48339

**XGBoost, Data Range: 2016-2021 Plain Features (Best Model Experiment)**
* CV Score: 0.61600
* LB Score: 0.46474

**XGBoost, Data Range: 2017-2021 Plain Features**
* CV Score: 0.62400
* LB Score: 0.51470

**XGBoost, Data Range: 2003-2021 Plain Features**
* CV Score: 0.57100
* LB Score: 0.55582

**XGBoost, Data Range: 2017-2021 Added Feature: PointsRatio**
* CV Score: 0.62100
* LB Score: 0.47907

**XGBoost, Data Range: 2015-2021 Plain Features**
* CV Score: 0.59400
* LB Score: 0.49742

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/Conferences.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySeedRoundSlots.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonDetailedResults.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MGameCities.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MSeasons.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MSecondaryTourneyTeams.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/Cities.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MTeamSpellings.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonCompactResults.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MMasseyOrdinals.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MSecondaryTourneyCompactResults.csv
/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourne

In [2]:
import re # Importing the Regular Expression Module...

In [3]:
%%time
# I like to disable my Notebook Warnings.
import warnings
warnings.filterwarnings('ignore')

CPU times: user 32 µs, sys: 4 µs, total: 36 µs
Wall time: 38.4 µs


In [4]:
%%time
# Notebook Configuration...

# Amount of data we want to load into the Model...
DATA_ROWS = None
# Dataframe, the amount of rows and cols to visualize...
NROWS = 50
NCOLS = 15
# Main data location path...
BASE_PATH = '...'

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 4.29 µs


In [5]:
%%time
# Configure notebook display settings to only use 2 decimal places, tables look nicer.
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', NCOLS) 
pd.set_option('display.max_rows', NROWS)

CPU times: user 75 µs, sys: 10 µs, total: 85 µs
Wall time: 87.7 µs


# Reading the Datasets

In [6]:
%%time
# Read the requiered datasets.
season_data = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MRegularSeasonCompactResults.csv')
seeds = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneySeeds.csv')

public_rating = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MMasseyOrdinals.csv')

CPU times: user 1.4 s, sys: 202 ms, total: 1.6 s
Wall time: 2.44 s


In [7]:
%%time
season_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 174471 entries, 0 to 174470
Data columns (total 8 columns):
 #   Column   Non-Null Count   Dtype 
---  ------   --------------   ----- 
 0   Season   174471 non-null  int64 
 1   DayNum   174471 non-null  int64 
 2   WTeamID  174471 non-null  int64 
 3   WScore   174471 non-null  int64 
 4   LTeamID  174471 non-null  int64 
 5   LScore   174471 non-null  int64 
 6   WLoc     174471 non-null  object
 7   NumOT    174471 non-null  int64 
dtypes: int64(7), object(1)
memory usage: 10.6+ MB
CPU times: user 30 ms, sys: 4.01 ms, total: 34 ms
Wall time: 45.7 ms


In [8]:
%%time
season_data.head()

CPU times: user 102 µs, sys: 13 µs, total: 115 µs
Wall time: 119 µs


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,20,1228,81,1328,64,N,0
1,1985,25,1106,77,1354,70,H,0
2,1985,25,1112,63,1223,56,H,0
3,1985,25,1165,70,1432,54,H,0
4,1985,25,1192,86,1447,74,H,0


In [9]:
%%time
season_data.describe()

CPU times: user 41.6 ms, sys: 3.08 ms, total: 44.7 ms
Wall time: 47.2 ms


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT
count,"174,471.00","174,471.00","174,471.00","174,471.00","174,471.00","174,471.00","174,471.00"
mean,"2,004.54",74.06,"1,287.16",76.76,"1,282.77",64.66,0.05
std,10.73,33.82,104.64,11.96,104.75,11.28,0.25
min,"1,985.00",0.00,"1,101.00",34.00,"1,101.00",20.00,0.00
25%,"1,995.00",47.00,"1,198.00",69.00,"1,191.00",57.00,0.00
50%,"2,005.00",76.00,"1,285.00",76.00,"1,281.00",64.00,0.00
75%,"2,014.00",103.00,"1,380.00",84.00,"1,375.00",72.00,0.00
max,"2,022.00",132.00,"1,472.00",186.00,"1,472.00",150.00,6.00


# Creating Features for the Model.

In [10]:
%%time
def score_gap(df):
    """
    """
    df['ScoreGap'] = df['WScore'] - df['LScore']
    return df

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.48 µs


In [11]:
%%time
season_data = score_gap(season_data)

CPU times: user 4.01 ms, sys: 131 µs, total: 4.14 ms
Wall time: 3.07 ms


In [12]:
%%time
season_data.head()

CPU times: user 107 µs, sys: 14 µs, total: 121 µs
Wall time: 126 µs


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT,ScoreGap
0,1985,20,1228,81,1328,64,N,0,17
1,1985,25,1106,77,1354,70,H,0,7
2,1985,25,1112,63,1223,56,H,0,7
3,1985,25,1165,70,1432,54,H,0,16
4,1985,25,1192,86,1447,74,H,0,12


In [13]:
%%time
# Creates an empty list of all the teams, Winners + Lossers to merge data back....
def create_team_list(df, group_list = ['Season', 'WTeamID'], team_id = 'WTeamID'):
    group = df.groupby(group_list).count().reset_index()
    group = group[group_list].rename(columns={team_id: "TeamID"})
    return group

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [14]:
%%time
winners = create_team_list(season_data, group_list = ['Season', 'WTeamID'], team_id = 'WTeamID')
lossers = create_team_list(season_data, group_list = ['Season', 'LTeamID'], team_id = 'LTeamID')

# Create an empty train dataset.
team_agg_features = pd.concat([winners, lossers], axis = 0).drop_duplicates().sort_values(['Season', 'TeamID']).reset_index(drop = True)

CPU times: user 84.5 ms, sys: 8.04 ms, total: 92.5 ms
Wall time: 95 ms


In [15]:
%%time
team_agg_features.head()

CPU times: user 109 µs, sys: 14 µs, total: 123 µs
Wall time: 127 µs


,Season,TeamID
0,1985,1102
1,1985,1103
2,1985,1104
3,1985,1106
4,1985,1108


In [16]:
%%time
# Creating aggregated features...
def winner_aggregated_features(df, group_list = ['Season', 'WTeamID']):
    tmp = df.groupby(group_list).agg(NumWins       = ('WTeamID', 'count'), 
                                     AvgWinsGap    = ('ScoreGap', 'mean'),
                                     W_TotalPoints = ('WScore', 'sum'),
                                     W_MaxPoints   = ('WScore', 'max'),
                                     W_MinPoints   = ('WScore', 'min'),
                                    )
    tmp = tmp.reset_index()
    tmp = tmp.rename(columns={"WTeamID": "TeamID"})
    
    return tmp

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 6.44 µs


In [17]:
%%time
# Creating aggregated features...
def losser_aggregated_features(df, group_list = ['Season', 'LTeamID']):
    tmp = df.groupby(group_list).agg(NumLosses       = ('LTeamID', 'count'), 
                                     AvgLossesGap    = ('ScoreGap', 'mean'),
                                     L_TotalPoints = ('LScore', 'sum'),
                                     L_MaxPoints   = ('LScore', 'max'),
                                     L_MinPoints   = ('LScore', 'min'),
                                    )
    tmp = tmp.reset_index()
    tmp = tmp.rename(columns={"LTeamID": "TeamID"})
    return tmp

CPU times: user 3 µs, sys: 1e+03 ns, total: 4 µs
Wall time: 6.2 µs


In [18]:
%%time
winner_team_aggregation = winner_aggregated_features(season_data)
losser_team_aggregation = losser_aggregated_features(season_data)

CPU times: user 60.5 ms, sys: 0 ns, total: 60.5 ms
Wall time: 67.3 ms


In [19]:
%%time
def merge_back(df):
    df = df.merge(winner_team_aggregation, on = ['Season', 'TeamID'], how = 'left')
    df = df.merge(losser_team_aggregation, on = ['Season', 'TeamID'], how = 'left')
    df.fillna(0, inplace = True) 
    return df

CPU times: user 2 µs, sys: 1e+03 ns, total: 3 µs
Wall time: 5.48 µs


In [20]:
%%time
team_agg_features = merge_back(team_agg_features)

CPU times: user 13.8 ms, sys: 0 ns, total: 13.8 ms
Wall time: 16.5 ms


In [21]:
%%time
def calculate_features(df):
    """
    
    """
    df['WinRatio'] = df['NumWins'] / (df['NumWins'] + df['NumLosses'])
    df['AvgScoreGap'] = ((df['NumWins'] * df['AvgWinsGap'] - df['NumLosses'] * df['AvgLossesGap']) / (df['NumWins'] + df['NumLosses']))
    df['PointsRatio'] = df['W_TotalPoints'] / (df['L_TotalPoints'] + df['W_TotalPoints'])
    return df

team_agg_features = calculate_features(team_agg_features)

CPU times: user 2.66 ms, sys: 0 ns, total: 2.66 ms
Wall time: 2.67 ms


In [22]:
%%time
team_agg_features = team_agg_features[['Season','TeamID','WinRatio', 'AvgScoreGap','PointsRatio']]

CPU times: user 3.2 ms, sys: 24 µs, total: 3.22 ms
Wall time: 2.69 ms


In [23]:
%%time
team_agg_features.head()

CPU times: user 113 µs, sys: 14 µs, total: 127 µs
Wall time: 131 µs


,Season,TeamID,WinRatio,AvgScoreGap,PointsRatio
0,1985,1102,0.21,-5.79,0.23
1,1985,1103,0.39,-3.04,0.45
2,1985,1104,0.70,7.80,0.74
3,1985,1106,0.42,-3.79,0.44
4,1985,1108,0.76,7.96,0.79


# Creating the Training Dataset...

In [24]:
%%time
tournament_data = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MNCAATourneyCompactResults.csv')
tournament_data.head()

CPU times: user 3.15 ms, sys: 1.84 ms, total: 4.98 ms
Wall time: 10.4 ms


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,WLoc,NumOT
0,1985,136,1116,63,1234,54,N,0
1,1985,136,1120,59,1345,58,N,0
2,1985,136,1207,68,1250,43,N,0
3,1985,136,1229,58,1425,55,N,0
4,1985,136,1242,49,1325,38,N,0


In [25]:
%%time
tournament_data.describe()

CPU times: user 12.7 ms, sys: 87 µs, total: 12.8 ms
Wall time: 12.3 ms


,Season,DayNum,WTeamID,WScore,LTeamID,LScore,NumOT
count,"2,317.00","2,317.00","2,317.00","2,317.00","2,317.00","2,317.00","2,317.00"
mean,"2,002.76",139.17,"1,286.54",76.89,"1,292.02",65.17,0.07
std,10.47,4.22,100.99,11.84,103.29,11.06,0.28
min,"1,985.00",134.00,"1,101.00",43.00,"1,101.00",29.00,0.00
25%,"1,994.00",136.00,"1,210.00",69.00,"1,209.00",58.00,0.00
50%,"2,003.00",137.00,"1,276.00",76.00,"1,290.00",65.00,0.00
75%,"2,012.00",139.00,"1,385.00",84.00,"1,387.00",72.00,0.00
max,"2,021.00",154.00,"1,463.00",149.00,"1,463.00",115.00,3.00


In [26]:
%%time
tournament_data = tournament_data.rename(columns = {'WTeamID' : 'W_TeamID', 'LTeamID' : 'L_TeamID', 'WScore' : 'W_Score', 'LScore' : 'L_Score'})  

CPU times: user 811 µs, sys: 0 ns, total: 811 µs
Wall time: 765 µs


In [27]:
%%time
# Drop non importante features from the dataset...
tournament_data.drop(['NumOT', 'WLoc'], axis = 1, inplace = True)

CPU times: user 822 µs, sys: 0 ns, total: 822 µs
Wall time: 828 µs


In [28]:
%%time
MIN_SEASON = 2015
# Remove data before 2003, no all the data is available...
tournament_data = tournament_data[tournament_data['Season'] >= MIN_SEASON].reset_index(drop = True)

CPU times: user 807 µs, sys: 992 µs, total: 1.8 ms
Wall time: 1.31 ms


In [29]:
%%time
tournament_data.head()

CPU times: user 0 ns, sys: 109 µs, total: 109 µs
Wall time: 113 µs


,Season,DayNum,W_TeamID,W_Score,L_TeamID,L_Score
0,2015,134,1214,74,1264,64
1,2015,134,1279,94,1140,90
2,2015,135,1173,56,1129,55
3,2015,135,1352,81,1316,77
4,2015,136,1112,93,1411,72


In [30]:
%%time
def merge_seed(df, seed_df, left_on = ['Season', 'W_TeamID'], field_name = 'SeedW'):
    df = pd.merge(df,seed_df, how = 'left', left_on = left_on, right_on = ['Season', 'TeamID'])
    df = df.drop('TeamID', axis = 1).rename(columns = {'Seed': field_name})
    return df

CPU times: user 3 µs, sys: 0 ns, total: 3 µs
Wall time: 5.96 µs


In [31]:
%%time
tournament_data = merge_seed(tournament_data, seeds, left_on = ['Season', 'W_TeamID'], field_name = 'W_Seed')
tournament_data = merge_seed(tournament_data, seeds, left_on = ['Season', 'L_TeamID'], field_name = 'L_Seed')

CPU times: user 6.88 ms, sys: 0 ns, total: 6.88 ms
Wall time: 6.89 ms


In [32]:
%%time
def seed_number(row):
    return int(re.sub("[^0-9]", "", row))

tournament_data['W_Seed'] = tournament_data['W_Seed'].apply(seed_number)
tournament_data['L_Seed'] = tournament_data['L_Seed'].apply(seed_number)

CPU times: user 1.7 ms, sys: 795 µs, total: 2.5 ms
Wall time: 2.5 ms


In [33]:
%%time
def merge_agg_features(df, agg_features):
    for result in ['W', 'L']:
        df = pd.merge(df, agg_features, how = 'left', left_on = ['Season', result +'_'+ 'TeamID'], right_on = ['Season', 'TeamID'])
        avoid = ['Season', 'TeamID']
        new_names = {col: result +'_'+ col for col in agg_features.columns if col not in avoid}
        df = df.rename(columns = new_names)        
        df = df.drop(columns = 'TeamID', axis = 1)
    return df

tournament_data = merge_agg_features(tournament_data, team_agg_features)

CPU times: user 7.77 ms, sys: 0 ns, total: 7.77 ms
Wall time: 7.78 ms


In [34]:
%%time
tournament_data.head()

CPU times: user 93 µs, sys: 13 µs, total: 106 µs
Wall time: 109 µs


,Season,DayNum,W_TeamID,W_Score,L_TeamID,L_Score,W_Seed,L_Seed,W_WinRatio,W_AvgScoreGap,W_PointsRatio,L_WinRatio,L_AvgScoreGap,L_PointsRatio
0,2015,134,1214,74,1264,64,16,16,0.48,-0.85,0.53,0.59,2.38,0.63
1,2015,134,1279,94,1140,90,11,11,0.62,5.12,0.64,0.72,8.91,0.74
2,2015,135,1173,56,1129,55,11,11,0.76,7.30,0.78,0.74,8.94,0.79
3,2015,135,1352,81,1316,77,16,16,0.58,1.27,0.62,0.65,5.42,0.68
4,2015,136,1112,93,1411,72,2,15,0.91,17.82,0.92,0.65,0.74,0.70


In [35]:
def replace_win_loser(df):
    team_a = df.copy()
    team_b = df.copy()
    
    team_a_dict, team_b_dict = {}, {}
    
    for col in team_a.columns:
        if col.find('W_') == 0:
            new_col_name = str(col).replace('W_', 'A_')
            team_a_dict[col] = new_col_name
        if col.find('L_') == 0:
            new_col_name = col.replace('L_', 'B_')    
            team_a_dict[col] = new_col_name
            
    for col in team_b.columns:
        if col.find('W_') == 0:
            new_col_name = str(col).replace('W_', 'B_')
            team_b_dict[col] = new_col_name
        if col.find('L_') == 0:
            new_col_name = col.replace('L_', 'A_')
            team_b_dict[col] = new_col_name

    team_a = team_a.rename(columns = team_a_dict)
    team_b = team_b.rename(columns = team_b_dict)
    
    merged_df = pd.concat([team_a, team_b], axis = 0, sort = False)
    return merged_df

In [36]:
%%time
tournament_data = replace_win_loser(tournament_data)

CPU times: user 2.04 ms, sys: 4 µs, total: 2.05 ms
Wall time: 2.1 ms


In [37]:
%%time
def calculate_differences(df):
    """
    
    """
    df['SeedDiff'] = df['A_Seed'] - df['B_Seed']
    df['WinRatioDiff'] = df['A_WinRatio'] - df['B_WinRatio']
    df['GapAvgDiff'] = df['A_AvgScoreGap'] - df['B_AvgScoreGap']    
    df['PointsRatioDiff'] = df['A_PointsRatio'] - df['A_PointsRatio']
    return df

tournament_data = calculate_differences(tournament_data)

CPU times: user 2.23 ms, sys: 29 µs, total: 2.26 ms
Wall time: 2.27 ms


---

# Creating the Target Variable

In [38]:
%%time
tournament_data['ScoreDiff'] = tournament_data['A_Score'] - tournament_data['B_Score']
tournament_data['A_Win'] = (tournament_data['ScoreDiff'] > 0).astype(int)
tournament_data = tournament_data.drop(columns=['A_Score', 'B_Score'])

CPU times: user 3.34 ms, sys: 0 ns, total: 3.34 ms
Wall time: 3.36 ms


In [39]:
%%time
tournament_data.head()

CPU times: user 110 µs, sys: 14 µs, total: 124 µs
Wall time: 128 µs


,Season,DayNum,A_TeamID,B_TeamID,A_Seed,B_Seed,A_WinRatio,...,B_PointsRatio,SeedDiff,WinRatioDiff,GapAvgDiff,PointsRatioDiff,ScoreDiff,A_Win
0,2015,134,1214,1264,16,16,0.48,...,0.63,0,-0.11,-3.22,0.00,10,1
1,2015,134,1279,1140,11,11,0.62,...,0.74,0,-0.09,-3.78,0.00,4,1
2,2015,135,1173,1129,11,11,0.76,...,0.79,0,0.02,-1.63,0.00,1,1
3,2015,135,1352,1316,16,16,0.58,...,0.68,0,-0.07,-4.15,0.00,4,1
4,2015,136,1112,1411,2,15,0.91,...,0.70,-13,0.26,17.09,0.00,21,1


In [40]:
tournament_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 802 entries, 0 to 400
Data columns (total 18 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   Season           802 non-null    int64  
 1   DayNum           802 non-null    int64  
 2   A_TeamID         802 non-null    int64  
 3   B_TeamID         802 non-null    int64  
 4   A_Seed           802 non-null    int64  
 5   B_Seed           802 non-null    int64  
 6   A_WinRatio       802 non-null    float64
 7   A_AvgScoreGap    802 non-null    float64
 8   A_PointsRatio    802 non-null    float64
 9   B_WinRatio       802 non-null    float64
 10  B_AvgScoreGap    802 non-null    float64
 11  B_PointsRatio    802 non-null    float64
 12  SeedDiff         802 non-null    int64  
 13  WinRatioDiff     802 non-null    float64
 14  GapAvgDiff       802 non-null    float64
 15  PointsRatioDiff  802 non-null    float64
 16  ScoreDiff        802 non-null    int64  
 17  A_Win           

---

# Creating the Test Dataset

In [41]:
%%time
sub_stage_one = pd.read_csv('/kaggle/input/mens-march-mania-2022/MDataFiles_Stage1/MSampleSubmissionStage1.csv')
tst_data = sub_stage_one.copy()

CPU times: user 10.2 ms, sys: 12 µs, total: 10.2 ms
Wall time: 17.1 ms


In [42]:
tst_data.shape

(11390, 2)

In [43]:
%%time
def separate_id(df):
    """
    
    """
    df['Season']  = df['ID'].apply(lambda x: int(x.split('_')[0]))
    df['TeamIdA'] = df['ID'].apply(lambda x: int(x.split('_')[1]))
    df['TeamIdB'] = df['ID'].apply(lambda x: int(x.split('_')[2]))
    return df

tst_data = separate_id(tst_data)

CPU times: user 24.7 ms, sys: 0 ns, total: 24.7 ms
Wall time: 24.7 ms


In [44]:
%%time
tst_data = merge_seed(tst_data, seeds, left_on = ['Season', 'TeamIdA'], field_name = 'A_Seed')
tst_data = merge_seed(tst_data, seeds, left_on = ['Season', 'TeamIdB'], field_name = 'B_Seed')

CPU times: user 16.6 ms, sys: 0 ns, total: 16.6 ms
Wall time: 18.5 ms


In [45]:
%%time
tst_data['A_Seed'] = tst_data['A_Seed'].apply(seed_number)
tst_data['B_Seed'] = tst_data['B_Seed'].apply(seed_number)

CPU times: user 39.4 ms, sys: 14 µs, total: 39.4 ms
Wall time: 39.4 ms


In [46]:
%%time
tst_data = tst_data.rename(columns = {'TeamIdA': 'A_TeamID', 'TeamIdB': 'B_TeamID'})

CPU times: user 1.01 ms, sys: 1e+03 ns, total: 1.02 ms
Wall time: 1.02 ms


In [47]:
tst_data

,ID,Pred,Season,A_TeamID,B_TeamID,A_Seed,B_Seed
0,2016_1112_1114,0.50,2016,1112,1114,6,12
1,2016_1112_1122,0.50,2016,1112,1122,6,16
2,2016_1112_1124,0.50,2016,1112,1124,6,5
3,2016_1112_1138,0.50,2016,1112,1138,6,14
4,2016_1112_1139,0.50,2016,1112,1139,6,9
...,...,...,...,...,...,...,...
11385,2021_1452_1457,0.50,2021,1452,1457,3,12
11386,2021_1452_1458,0.50,2021,1452,1458,3,9
11387,2021_1455_1457,0.50,2021,1455,1457,11,12
11388,2021_1455_1458,0.50,2021,1455,1458,11,9


In [48]:
team_agg_features

,Season,TeamID,WinRatio,AvgScoreGap,PointsRatio
0,1985,1102,0.21,-5.79,0.23
1,1985,1103,0.39,-3.04,0.45
2,1985,1104,0.70,7.80,0.74
3,1985,1106,0.42,-3.79,0.44
4,1985,1108,0.76,7.96,0.79
...,...,...,...,...,...
12294,2022,1468,0.45,-4.30,0.50
12295,2022,1469,0.37,-8.00,0.42
12296,2022,1470,0.38,-3.05,0.42
12297,2022,1471,0.40,-4.00,0.47


In [49]:
%%time
def merge_agg_features(df, agg_features):
    for result in ['A', 'B']:
        df = pd.merge(df, agg_features, how = 'left', left_on = ['Season', result +'_'+ 'TeamID'], right_on = ['Season', 'TeamID'])
        avoid = ['Season', 'TeamID']
        new_names = {col: result +'_'+ col for col in agg_features.columns if col not in avoid}
        df = df.rename(columns = new_names)        
        df = df.drop(columns = 'TeamID', axis = 1)
    return df

tst_data = merge_agg_features(tst_data, team_agg_features)

CPU times: user 14.3 ms, sys: 25 µs, total: 14.3 ms
Wall time: 14.3 ms


In [50]:
%%time
tst_data = calculate_differences(tst_data)

CPU times: user 2.91 ms, sys: 0 ns, total: 2.91 ms
Wall time: 2.93 ms


In [51]:
%%time
tst_data

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.01 µs


,ID,Pred,Season,A_TeamID,B_TeamID,A_Seed,B_Seed,...,B_WinRatio,B_AvgScoreGap,B_PointsRatio,SeedDiff,WinRatioDiff,GapAvgDiff,PointsRatioDiff
0,2016_1112_1114,0.50,2016,1112,1114,6,12,...,0.87,9.94,0.88,-6,-0.11,2.28,0.00
1,2016_1112_1122,0.50,2016,1112,1122,6,16,...,0.48,-2.36,0.54,-10,0.27,14.58,0.00
2,2016_1112_1124,0.50,2016,1112,1124,6,5,...,0.66,6.69,0.70,1,0.10,5.52,0.00
3,2016_1112_1138,0.50,2016,1112,1138,6,14,...,0.58,0.67,0.62,-8,0.18,11.55,0.00
4,2016_1112_1139,0.50,2016,1112,1139,6,9,...,0.68,9.42,0.74,-3,0.08,2.79,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11385,2021_1452_1457,0.50,2021,1452,1457,3,12,...,0.96,12.75,0.97,-9,-0.29,-7.56,0.00
11386,2021_1452_1458,0.50,2021,1452,1458,3,9,...,0.59,5.31,0.62,-6,0.08,-0.13,0.00
11387,2021_1455_1457,0.50,2021,1455,1457,11,12,...,0.96,12.75,0.97,-1,-0.22,-10.12,0.00
11388,2021_1455_1458,0.50,2021,1455,1458,11,9,...,0.59,5.31,0.62,2,0.15,-2.68,0.00


In [52]:
%%time
tst_data.shape

CPU times: user 14 µs, sys: 2 µs, total: 16 µs
Wall time: 18.8 µs


(11390, 17)

# Building the Model...
Ok, so up to this point we have been aggregating and merging data from the Seasons Datasets...

* We used the season data to create aggregated features by team and season. and merge this back to the tournament data
* Using the merged data we calculate the outcomes of each of the games, Win or Lost.
* This new calculated variable will be our target.

In [53]:
%%time
from sklearn import tree
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.metrics import log_loss
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier

CPU times: user 1.28 s, sys: 518 ms, total: 1.8 s
Wall time: 2.62 s


In [54]:
%%time
target_feature = 'A_Win'
avoid = ['ScoreDiff', 'Season', 'DayNum', 'A_Win']
features = [col for col in tournament_data.columns if col not in avoid]

CPU times: user 29 µs, sys: 4 µs, total: 33 µs
Wall time: 37.2 µs


In [55]:
%%time
features

CPU times: user 2 µs, sys: 0 ns, total: 2 µs
Wall time: 5.72 µs


['A_TeamID',
 'B_TeamID',
 'A_Seed',
 'B_Seed',
 'A_WinRatio',
 'A_AvgScoreGap',
 'A_PointsRatio',
 'B_WinRatio',
 'B_AvgScoreGap',
 'B_PointsRatio',
 'SeedDiff',
 'WinRatioDiff',
 'GapAvgDiff',
 'PointsRatioDiff']

In [56]:
features = ['A_TeamID',
            'B_TeamID',
            'A_Seed',
            'B_Seed',
            'A_WinRatio',
            'A_AvgScoreGap',
            'A_PointsRatio',
            'B_WinRatio',
            'B_AvgScoreGap',
            'B_PointsRatio',
            'SeedDiff',
            'WinRatioDiff',
            'GapAvgDiff',
            'PointsRatioDiff',
           ]

In [57]:
%%time
# Develop a CV loop to avoid leaking data from future tournaments...
def kfold_model(train_df, tst_df):
    cvs = []
    preds_test = []
    seasons = train_df['Season'].unique()
    
    for season in seasons[1:]:
        print(f'\nValidating on season {season}')
        X_train = train_df[train_df['Season'] < season][features].reset_index(drop = True).copy()
        X_val = train_df[train_df['Season'] == season][features].reset_index(drop = True).copy()
        
        y_train = train_df[train_df['Season'] < season][target_feature].reset_index(drop = True).copy()
        y_val = train_df[train_df['Season'] == season][target_feature].reset_index(drop = True).copy()
        
        tst_dataset = tst_data[features].copy()
        
        
        scaler = MinMaxScaler()
        scaler.fit(X_train)
        
        X_train = scaler.transform(X_train)        
        X_val = scaler.transform(X_val)
        tst_dataset = scaler.transform(tst_dataset)
        
        model = XGBClassifier(n_estimators = 1024, random_state = 85)
        model.fit(X_train, y_train, eval_set = [(X_val, y_val)], verbose = 0, early_stopping_rounds = 128)
        pred = model.predict_proba(X_val)[:, 1]
        
        pred_test = model.predict_proba(tst_dataset)[:, 1]
        preds_test.append(pred_test)
        
        loss = log_loss(y_val, pred)
        cvs.append(loss)
        
        print(f'\t -> Scored {loss:.4f}')
    print(f'\nLocal Cross Validation Score Is: {np.mean(cvs):.3f}', '\n')
    return preds_test

CPU times: user 4 µs, sys: 0 ns, total: 4 µs
Wall time: 7.39 µs


In [58]:
%%time
predictions = kfold_model(tournament_data, tournament_data)


Validating on season 2016
[22:45:42] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
	 -> Scored 0.6392

Validating on season 2017
[22:45:43] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
	 -> Scored 0.5708

Validating on season 2018
[22:45:44] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
	 -> Scored 0.6114

Validating on season 2019
[22:45:45] WARNING: ../src/learner.cc:1115: Starting in XGBoost 1.3.0, the defa

In [59]:
# Local Cross Validation Score Is: 0.616 
# Local Cross Validation Score Is: 0.608...

In [60]:
%%time
mean_predictions = np.mean(predictions, 0)

sub = tst_data[['ID', 'Pred']].copy()
sub['Pred'] = mean_predictions
sub.to_csv('submission_02242022.csv', index = False)

CPU times: user 23.6 ms, sys: 40 µs, total: 23.7 ms
Wall time: 23.7 ms


---